In [1]:
import numpy as np
import matplotlib.pyplot as plt
from particle.mayaviOffScreen import mlab
from particle.pipeline import Sand
mlab.init_notebook()

Set mlab.options.offscreen=True
Notebook initialized with ipy backend.


In [2]:
data = np.load("data/liutao/v1/particles.npz")["testSet"]
print(data.shape, data.max())

(6786, 1, 64, 64, 64) 255


In [3]:
sand = Sand(data[0,0])
sand.visualize()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [4]:
coords = sand.toCoords().T
sand.pca()

(array([37.57261385, 17.31241208, 12.2242338 ]),
 array([[ 0.73605909, -0.66693909, -0.11579841],
        [-0.64493442, -0.63898306, -0.4192377 ],
        [-0.20561279, -0.3832661 ,  0.90046126]]))

In [5]:
covarMatrix = np.cov(sand.toCoords().T)
# np.linalg.eig(square_array) 返回输入方阵的特征值和右特征向量
eigenvalues, eigenvectors = np.linalg.eig(covarMatrix)
order = np.argsort(eigenvalues)
eigenvalues, eigenvectors

(array([37.57261385, 17.31241208, 12.2242338 ]),
 array([[ 0.73605909, -0.64493442, -0.20561279],
        [-0.66693909, -0.63898306, -0.3832661 ],
        [-0.11579841, -0.4192377 ,  0.90046126]]))

In [6]:
from sklearn.decomposition import PCA
pcaModel = PCA(n_components=3).fit(coords.T)
newPoints = pcaModel.transform(coords.T)
pcaModel.components_, pcaModel.explained_variance_

(array([[-0.73605909,  0.66693909,  0.11579841],
        [ 0.64493442,  0.63898306,  0.4192377 ],
        [-0.20561279, -0.3832661 ,  0.90046126]]),
 array([37.57261385, 17.31241208, 12.2242338 ]))

In [7]:
sand.visualize(sand.poseNormalization())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [8]:
(np.dot(coords.T-coords.T.mean(axis=0), pcaModel.components_.T) == newPoints).all()

True

In [9]:
newPoints.shape

(3371, 3)

In [10]:
newPoints += 32

In [11]:
from scipy.interpolate import griddata
pointGrid = np.zeros((64, 64, 64), dtype=np.uint8)
pointGridTmp = np.where(pointGrid == 0)
pointGridTmp = np.asarray(pointGridTmp).T
grid = griddata(newPoints, np.ones(len(newPoints)), pointGridTmp, method="linear", fill_value=0)
pointGrid[pointGridTmp[:, 0], pointGridTmp[:, 1], pointGridTmp[:, 2]] = grid
sand.visualize(pointGrid)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [13]:
np.dot([1, 1, 1], pcaModel.components_.T)

array([0.04667841, 1.70315518, 0.31158237])

In [14]:
pcaModel.components_.T

array([[-0.73605909,  0.64493442, -0.20561279],
       [ 0.66693909,  0.63898306, -0.3832661 ],
       [ 0.11579841,  0.4192377 ,  0.90046126]])

In [15]:
from skimage import transform
sand.visualize(transform.resize(sand.cube, (128,)*3))

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [16]:
np.unique(transform.resize(sand.cube, (128,)*3).ravel())

array([0.      , 0.015625, 0.046875, 0.0625  , 0.09375 , 0.109375,
       0.140625, 0.15625 , 0.1875  , 0.203125, 0.234375, 0.25    ,
       0.28125 , 0.296875, 0.328125, 0.34375 , 0.375   , 0.390625,
       0.421875, 0.4375  , 0.46875 , 0.515625, 0.53125 , 0.5625  ,
       0.578125, 0.609375, 0.625   , 0.65625 , 0.671875, 0.703125,
       0.71875 , 0.75    , 0.765625, 0.796875, 0.8125  , 0.84375 ,
       0.859375, 0.890625, 0.90625 , 0.9375  , 0.953125, 0.984375,
       1.      ])

In [17]:
np.unique(transform.rescale(sand.cube, 2))

array([0.      , 0.015625, 0.046875, 0.0625  , 0.09375 , 0.109375,
       0.140625, 0.15625 , 0.1875  , 0.203125, 0.234375, 0.25    ,
       0.28125 , 0.296875, 0.328125, 0.34375 , 0.375   , 0.390625,
       0.421875, 0.4375  , 0.46875 , 0.515625, 0.53125 , 0.5625  ,
       0.578125, 0.609375, 0.625   , 0.65625 , 0.671875, 0.703125,
       0.71875 , 0.75    , 0.765625, 0.796875, 0.8125  , 0.84375 ,
       0.859375, 0.890625, 0.90625 , 0.9375  , 0.953125, 0.984375,
       1.      ])